In [1]:
import cv2
import os
import glob
import json
import time
from datetime import datetime
import matplotlib.pyplot as plt


In [3]:
class FaceDetectionPipeline:
    def __init__(self):
        self.detector = cv2.CascadeClassifier(
            cv2.data.haarcascades + "haarcascade_frontalface_default.xml"
        )

    def process_image(self, img_path):
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.detector.detectMultiScale(gray, 1.3, 5)
        return faces, None, img

In [ ]:
def detect_folder(folder="test_images", out_dir="output_images"):
    os.makedirs(out_dir, exist_ok=True)
    pipeline = FaceDetectionPipeline()

    images = []
    for ext in ("*.jpg", "*.png", "*.jpeg", "*.bmp"):
        images.extend(glob.glob(os.path.join(folder, ext)))

    print(f"Tìm thấy {len(images)} ảnh")

    for img_path in images:
        faces, _, img = pipeline.process_image(img_path)
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        out_path = os.path.join(out_dir, os.path.basename(img_path))
        cv2.imwrite(out_path, img)

    print("Hoàn tất detect folder")


In [9]:
def create_gt_template(folder="test_images", output="ground_truth.json"):
    images = []
    for ext in ("*.jpg", "*.png", "*.jpeg", "*.bmp"):
        images.extend(glob.glob(os.path.join(folder, ext)))

    data = [{"image": os.path.basename(p), "faces": []} for p in images]

    with open(output, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print(f"Đã tạo GT template: {output}")


In [ ]:
create_gt_template()


In [10]:
def auto_fill_gt(img_dir="output_images", gt_file="ground_truth.json"):
    pipeline = FaceDetectionPipeline()

    with open(gt_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    for item in data:
        img_path = os.path.join(img_dir, item["image"])
        if not os.path.exists(img_path):
            print(f"⚠ Không thấy {img_path}")
            continue

        faces, _, _ = pipeline.process_image(img_path)
        item["faces"] = [[int(x), int(y), int(w), int(h)] for x,y,w,h in faces]

    with open(gt_file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

    print("Đã auto-fill ground truth")


In [ ]:
auto_fill_gt()

In [11]:
def iou(b1, b2):
    x1,y1,w1,h1 = b1
    x2,y2,w2,h2 = b2

    xi1 = max(x1, x2)
    yi1 = max(y1, y2)
    xi2 = min(x1+w1, x2+w2)
    yi2 = min(y1+h1, y2+h2)

    inter = max(0, xi2-xi1) * max(0, yi2-yi1)
    union = w1*h1 + w2*h2 - inter
    return inter/union if union else 0


In [14]:
def evaluate(folder="test_images", gt_file="ground_truth.json"):
    pipeline = FaceDetectionPipeline()

    with open(gt_file, "r", encoding="utf-8") as f:
        data = json.load(f)

    TP = FP = FN = 0
    start = time.time()

    for item in data:
        img_path = os.path.join(folder, item["image"])
        if not os.path.exists(img_path):
            continue

        preds, _, _ = pipeline.process_image(img_path)
        gts = item["faces"]
        matched = set()

        for p in preds:
            best = 0
            idx = -1
            for i,g in enumerate(gts):
                if i in matched: continue
                v = iou(p, g)
                if v > best:
                    best = v
                    idx = i
            if best >= 0.5:
                TP += 1
                matched.add(idx)
            else:
                FP += 1

        FN += len(gts) - len(matched)

    precision = TP/(TP+FP) if TP+FP else 0
    recall = TP/(TP+FN) if TP+FN else 0
    f1 = 2*precision*recall/(precision+recall) if precision+recall else 0

    print("KẾT QUẢ ĐÁNH GIÁ")
    print(f"TP={TP}, FP={FP}, FN={FN}")
    print(f"Precision={precision:.4f}")
    print(f"Recall={recall:.4f}")
    print(f"F1-score={f1:.4f}")
    print(f"Time: {time.time()-start:.2f}s")

    return precision, recall, f1

In [15]:
evaluate()

KẾT QUẢ ĐÁNH GIÁ
TP=13542, FP=793, FN=7204
Precision=0.9447
Recall=0.6528
F1-score=0.7720
Time: 412.22s


(0.9446808510638298, 0.6527523378000578, 0.7720418460135116)